## Human in the Loop🤵

In [8]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Image, display

In [4]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


class State(TypedDict):
    messages: Annotated[list, add_messages]

GENERATE_POST = "generate_post"
GET_REVIEW_DECISION = "get_review_decision"
POST = "post"
COLLECT_FEEDBACK = "collect_feedback"

In [6]:
def generate_post(state : State):
    return {"messages": [llm.invoke(state["messages"])]}


def get_review_decision(state: State):
    post_content = state["messages"][-1].content
    print("\nCurrent LinkedIn Post\n")
    print(post_content)
    print("\n")

    decision = input("👁️Are you satisfied with this suggestion? (Yes/No): ")

    if decision.lower() == "yes":
        return POST
    else:
        return COLLECT_FEEDBACK
    
def post(state: State):
    final_post = state["messages"][-1].content
    print("\🗣️Final LinkedIn Post\n")
    print(final_post)
    print("\n👍 Post approved 🥂")

def collect_feedback(state: State):
    feedback = input("☹️How can I improve this post?")
    return {"messages": [HumanMessage(content=feedback)]}

In [13]:
graph = StateGraph(State)

graph.add_node(GENERATE_POST, generate_post)
graph.add_node(COLLECT_FEEDBACK, collect_feedback)
graph.add_node(POST, post)

graph.set_entry_point(GENERATE_POST)
graph.add_conditional_edges(GENERATE_POST, get_review_decision, {POST:"post", COLLECT_FEEDBACK: "collect_feedback"})
graph.add_edge(COLLECT_FEEDBACK, GENERATE_POST)
graph.add_edge(POST, END)

app = graph.compile()

# display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({"messages": [HumanMessage(content="Write me a linked in post about Integration into software engineering")]})
print(result)